In [54]:
import requests
import lxml.html
import pandas as pd
import numpy as np

In [12]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [24]:
doc = lxml.html.fromstring(website_url.content)

In [14]:
tree = lxml.html.fromstring(website_url.text)

In [25]:
tr_elements = doc.xpath('//tr')

In [26]:
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [27]:
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print '%d:"%s"'%(i,name)
    col.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighbourhood
"


In [28]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [29]:
[len(C) for (title,C) in col]

[288, 288, 288]

In [99]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [67]:
df.head()

,Borough,Neighbourhood,Postcode
0,Not assigned,Not assigned\n,M1A
1,Not assigned,Not assigned\n,M2A
2,North York,Parkwoods\n,M3A
3,North York,Victoria Village\n,M4A
4,Downtown Toronto,Harbourfront\n,M5A


In [68]:
c = df.columns[1]

In [44]:
c

'Neighbourhood\n'

In [100]:
df = df[['Postcode', 'Borough', c]]

In [101]:
df['Neighborhood'] = df[c]

In [102]:
df.drop([c],axis=1,inplace=True)

In [72]:
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [103]:
df['Neighborhood'].replace('Not assigned\n', np.nan,inplace=True)

In [104]:
df.at[8 , 'Neighborhood'] = "Queen's Park"

In [105]:
df.head(10)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n
7,M6A,North York,Lawrence Manor\n
8,M7A,Queen's Park,Queen's Park
9,M8A,Not assigned,NaN


In [106]:
df.dropna(inplace=True)

In [107]:
df.head(10)

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n
7,M6A,North York,Lawrence Manor\n
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue\n
11,M1B,Scarborough,Rouge\n
12,M1B,Scarborough,Malvern\n


In [108]:
df = df.groupby(['Postcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

In [112]:
df.head(20)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge\n, Malvern\n"
1,M1C,Scarborough,"Highland Creek\n, Rouge Hill\n, Port Union\n"
2,M1E,Scarborough,"Guildwood\n, Morningside\n, West Hill\n"
3,M1G,Scarborough,Woburn\n
4,M1H,Scarborough,Cedarbrae\n
5,M1J,Scarborough,Scarborough Village\n
6,M1K,Scarborough,"East Birchmount Park\n, Ionview\n, Kennedy Park\n"
7,M1L,Scarborough,"Clairlea\n, Golden Mile\n, Oakridge\n"
8,M1M,Scarborough,"Cliffcrest\n, Cliffside\n, Scarborough Village..."
9,M1N,Scarborough,"Birch Cliff\n, Cliffside West\n"


I had some issues trying to use BeatifulSoup so I used xlml.html. I got a prrety result I guess, and after reading a while about how to join duplicated postcodes I got the data frame listed above

In [114]:
df_tor = pd.read_csv('Geospatial_Coordinates.csv')

In [115]:
df_tor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [119]:
df.join(df_tor.set_index('Postal Code'), on='Postcode')

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge\n, Malvern\n",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek\n, Rouge Hill\n, Port Union\n",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood\n, Morningside\n, West Hill\n",43.763573,-79.188711
3,M1G,Scarborough,Woburn\n,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae\n,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village\n,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park\n, Ionview\n, Kennedy Park\n",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea\n, Golden Mile\n, Oakridge\n",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest\n, Cliffside\n, Scarborough Village...",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff\n, Cliffside West\n",43.692657,-79.264848
